In [ ]:
!pip install -q librosa soundfile awscli boto3

print("✅ All libraries installed successfully!")

## 2. Configure AWS S3 Access

**Add secrets in Kaggle:**
1. Settings → Add-ons → Secrets
2. Add: `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION`

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

# Load AWS credentials from Kaggle Secrets
user_secrets = UserSecretsClient()

try:
    os.environ['AWS_ACCESS_KEY_ID'] = user_secrets.get_secret('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = user_secrets.get_secret('AWS_SECRET_ACCESS_KEY')
    os.environ['AWS_DEFAULT_REGION'] = user_secrets.get_secret('AWS_REGION')
    print("✅ AWS credentials loaded from Kaggle secrets")
except:
    print("⚠️  Kaggle secrets not found. Add them in Settings → Secrets")
    raise

# Verify AWS access
!aws s3 ls s3://alertreck/

## 3. Download Preprocessed Data from S3

**Note:** With 30GB RAM, we can download the FULL dataset directly - no chunking needed!

In [ ]:
# Create working directory
!mkdir -p /kaggle/working/preprocessed_data
!mkdir -p /kaggle/working/train_chunks

S3_BUCKET = "alertreck"
DATA_DIR = "/kaggle/working/preprocessed_data"

print("📥 Downloading preprocessed data from S3...")
print("⚠️  Using chunked training data to avoid Kaggle's 20GB disk limit")
print("Files: train_chunks (10x ~2GB), val_data.pkl (960MB), test_data.pkl (1.1GB)")
print("⏰ This may take 10-15 minutes depending on connection speed.\n")

# Download chunked training data (saves disk space!)
print("Downloading training chunks...")
!aws s3 sync s3://{S3_BUCKET}/preprocessed_data/train_chunks/ /kaggle/working/train_chunks/

# Download val, test, and config
print("\nDownloading validation and test data...")
!aws s3 cp s3://{S3_BUCKET}/preprocessed_data/val_data.pkl {DATA_DIR}/val_data.pkl
!aws s3 cp s3://{S3_BUCKET}/preprocessed_data/test_data.pkl {DATA_DIR}/test_data.pkl
!aws s3 cp s3://{S3_BUCKET}/preprocessed_data/preprocessing_config.json {DATA_DIR}/preprocessing_config.json

print("\n✅ All data downloaded!")

# Verify downloads
print("\n📁 Downloaded files:")
!ls -lh {DATA_DIR}

# Load configuration
import json
with open(f'{DATA_DIR}/preprocessing_config.json', 'r') as f:
    config = json.load(f)

print(f"\n📊 Dataset Summary:")
print(f"  Total files processed: {config['dataset_stats']['total_files']:,}")
print(f"  Training samples: {config['dataset_stats']['train_size']:,}")
print(f"  Validation samples: {config['dataset_stats']['val_size']:,}")
print(f"  Test samples: {config['dataset_stats']['test_size']:,}")
print(f"\n🎵 Audio Configuration:")
print(f"  Sample rate: {config['target_sr']} Hz")
print(f"  Duration: {config['duration']} seconds")
print(f"  Mel bands: {config['n_mels']}")

## 4. Load Data into Memory

**Kaggle Advantage:** With 30GB RAM, we load everything at once - much simpler!

In [ ]:
import pickle
import numpy as np
import tensorflow as tf
import gc

print("📂 Loading FULL datasets into RAM (no chunking)...\n")

# Load training data from chunks
print("Loading training chunks...")
import glob
train_data = []
chunk_files = sorted(glob.glob('/kaggle/working/train_chunks/train_chunk_*.pkl'))
for chunk_file in chunk_files:
    with open(chunk_file, 'rb') as f:
        chunk = pickle.load(f)
        train_data.extend(chunk)
        del chunk
        gc.collect()

print(f"✅ Loaded {len(train_data):,} training samples")

# Load validation data
with open(f'{DATA_DIR}/val_data.pkl', 'rb') as f:
    val_data = pickle.load(f)
print(f"✅ Loaded {len(val_data):,} validation samples")

# Extract features and labels
print("\nExtracting features and labels...")
X_train = np.array([sample['features']['mel_spectrogram'] for sample in train_data], dtype=np.float32)
y_train = np.array([sample['label']['threat_level'] for sample in train_data], dtype=np.int32)

X_val = np.array([sample['features']['mel_spectrogram'] for sample in val_data], dtype=np.float32)
y_val = np.array([sample['label']['threat_level'] for sample in val_data], dtype=np.int32)

# Free memory
del train_data, val_data
gc.collect()

print(f"\n📊 Dataset shapes:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}")
print(f"  y_val: {y_val.shape}")

# Create tf.data.Dataset for optimized training
BATCH_SIZE = 64  # Can try 128 if GPU memory allows

print(f"\n🔄 Creating optimized tf.data.Dataset pipelines...")
print(f"  Batch size: {BATCH_SIZE}")

# Preprocessing function with augmentation for custom CNN
def preprocess_with_augmentation(x, y, augment=False):
    # Add channel dimension: (128, 431) -> (128, 431, 1)
    x = tf.expand_dims(x, axis=-1)
    
    # Data augmentation for training (reduces overfitting)
    if augment:
        # Random amplitude scaling (simulates volume variations)
        scale = tf.random.uniform([], minval=0.8, maxval=1.2)
        x = x * scale
        
        # Add slight Gaussian noise (simulates background noise)
        noise = tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=0.05)
        x = x + noise
        
        # Random time shift (simulates different event timing)
        shift = tf.random.uniform([], minval=-10, maxval=10, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=1)
        
        x = tf.clip_by_value(x, -3.0, 3.0)
    
    return x, y

# Wrapper functions for augmented vs clean preprocessing
def preprocess_train(x, y):
    return preprocess_with_augmentation(x, y, augment=True)

def preprocess_val(x, y):
    return preprocess_with_augmentation(x, y, augment=False)

# Training dataset with augmentation
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train), reshuffle_each_iteration=True)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.map(preprocess_train, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)  # Don't cache augmented data

# Validation dataset (no augmentation)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.map(preprocess_val, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache()  # Cache in RAM for speed
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

print(f"\n✅ Datasets created with caching and prefetching!")
print(f"  Training batches: {len(X_train) // BATCH_SIZE}")
print(f"  Validation batches: {len(X_val) // BATCH_SIZE}")
print(f"  🚀 Ready for ultra-fast training with Custom CNN!")

## 5. Prepare Data for Training

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

class_names = ['BACKGROUND', 'THREAT_CONTEXT', 'THREAT']

print("Computing class weights from training data...")

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("\nClass weights (for balanced training):")
for cls, weight in class_weight_dict.items():
    count = np.sum(y_train == cls)
    print(f"  {class_names[cls]}: {weight:.3f} (n={count:,})")

print(f"\nCustom CNN input shape: (128, 431, 1)")
print("✅ Ready for custom CNN training!")

## 6. Build Custom CNN Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

# IMPORTANT: Configure GPU settings BEFORE any TensorFlow operations
# Enable GPU memory growth FIRST (before runtime initialization)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU memory growth enabled")
    except RuntimeError as e:
        print(f"⚠️  Could not set memory growth (already initialized): {e}")

# Now enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("✅ Mixed precision (float16) enabled for faster training")

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

print("\n🚀 Building Custom CNN Model for Audio Classification...")

def build_custom_cnn(input_shape=(128, 431, 1), num_classes=3):
    """Build custom CNN model optimized for mel-spectrogram threat detection."""
    
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # Block 1: Extract low-level features
        layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Block 2: Mid-level features
        layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Block 3: High-level features
        layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Block 4: Deep features
        layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                     kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.5),
        
        # Classification head
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu',
                    kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu',
                    kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax', dtype='float32')  # float32 for stability
    ])
    
    return model

# Build model
model = build_custom_cnn(input_shape=(128, 431, 1), num_classes=3)
model.summary()

print(f"\n📊 Model parameters: {model.count_params():,}")
print("💡 Custom CNN optimized for mel-spectrogram features")

# Compile with Adam optimizer
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n✅ Model compiled!")
print("   4 conv blocks + strong regularization (L2, Dropout, BatchNorm)")
print("   Optimized for audio spectrogram classification!")

## 7. Setup Training Callbacks

In [ ]:
from tensorflow.keras import callbacks

# Create model directory
!mkdir -p /kaggle/working/models

# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = callbacks.ModelCheckpoint(
    filepath='/kaggle/working/models/best_model_custom_cnn.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-8,
    verbose=1
)

callback_list = [early_stopping, model_checkpoint, reduce_lr]

print("✅ Callbacks configured (weights-only checkpoint)")

## 8. Train Model

In [ ]:
print("🚀 Starting Custom CNN Training...\n")
print(f"💡 Batch size: {BATCH_SIZE}")
print("💾 Full dataset cached in RAM for maximum speed")
print("🎯 Class weighting enabled for balanced training")
print("⚡ Mixed precision + GPU acceleration\n")

print(f"📊 Dataset info:")
print(f"  Training samples: {len(X_train):,}")
print(f"  Validation samples: {len(X_val):,}")
print(f"  Training batches per epoch: {len(X_train) // BATCH_SIZE}")
print(f"  Validation batches per epoch: {len(X_val) // BATCH_SIZE}\n")

print("⏳ Expected training time: 15-30 minutes with GPU...\n")

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    class_weight=class_weight_dict,
    callbacks=callback_list,
    verbose=1
)

print("\n✅ Training complete!")
print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

## 9. Plot Training History

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Validation')
axes[0].set_title('Model Loss (Custom CNN)', fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train')
axes[1].plot(history.history['val_accuracy'], label='Validation')
axes[1].set_title('Model Accuracy (Custom CNN)', fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Evaluate Model

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

print("📊 Evaluating on validation set...\n")

# Evaluate on validation set
val_results = model.evaluate(val_dataset, verbose=1)

print("\nValidation Results:")
print(f"  Loss: {val_results[0]:.4f}")
print(f"  Accuracy: {val_results[1]:.4f}")

# Get predictions
print("\nGenerating predictions...")
y_pred_proba = model.predict(val_dataset, verbose=1)
y_pred = np.argmax(y_pred_proba, axis=1)

# Classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted', fontweight='bold')
plt.ylabel('True', fontweight='bold')
plt.title('Confusion Matrix - Custom CNN - Validation Set', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

print("\n✅ Evaluation complete!")
test_results = val_results  # For compatibility with save cell

## 11. Save Model and Export to TFLite

In [ ]:
# Save full model
model.save('/kaggle/working/models/threat_detection_custom_cnn.keras')
print("✅ Full model saved")

# Load best weights
model.load_weights('/kaggle/working/models/best_model_custom_cnn.weights.h5')
print("✅ Loaded best weights from checkpoint")

# Export to TensorFlow Lite
print("\nExporting to TensorFlow Lite...")
print("Converting mixed precision model to float32 for TFLite compatibility...")

# Create a float32 version of the model for TFLite conversion
tf.keras.mixed_precision.set_global_policy('float32')
model_f32 = build_custom_cnn(input_shape=(128, 431, 1), num_classes=3)
model_f32.set_weights(model.get_weights())  # Copy weights from mixed precision model
print("✅ Created float32 model for conversion")

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_f32)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]  # Quantize to float16 for smaller size
tflite_model = converter.convert()

with open('/kaggle/working/models/threat_detection_custom_cnn.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"✅ TensorFlow Lite model: {len(tflite_model) / 1024 / 1024:.1f} MB")

# Save model configuration
model_config = {
    'model_type': 'CustomCNN',
    'val_accuracy': float(test_results[1]) if test_results else None,
    'val_loss': float(test_results[0]) if test_results else None,
    'class_names': class_names,
    'input_shape': [128, 431, 1],
    'preprocessing': config,
    'batch_size': BATCH_SIZE,
    'total_parameters': int(model.count_params())
}

import json
with open('/kaggle/working/models/model_config_custom_cnn.json', 'w') as f:
    json.dump(model_config, f, indent=2)

print("✅ Model configuration saved")

# Upload to S3
print("\nUploading models to S3...")
!aws s3 cp /kaggle/working/models/threat_detection_custom_cnn.keras s3://{S3_BUCKET}/models/custom_cnn/
!aws s3 cp /kaggle/working/models/best_model_custom_cnn.weights.h5 s3://{S3_BUCKET}/models/custom_cnn/
!aws s3 cp /kaggle/working/models/threat_detection_custom_cnn.tflite s3://{S3_BUCKET}/models/custom_cnn/
!aws s3 cp /kaggle/working/models/model_config_custom_cnn.json s3://{S3_BUCKET}/models/custom_cnn/

print("\n✅ Models uploaded to S3!")
print(f"   Location: s3://{S3_BUCKET}/models/custom_cnn/")
print("\n📦 Files uploaded:")
print("  - threat_detection_custom_cnn.keras (full model)")
print("  - best_model_custom_cnn.weights.h5 (best weights)")
print("  - threat_detection_custom_cnn.tflite (edge deployment, optimized)")
print("  - model_config_custom_cnn.json (configuration)")

## Summary

### Custom CNN Training Complete! 🎉

**Model Architecture:**
- 4 convolutional blocks (32→64→128→256 filters)
- Strong regularization: L2, Dropout, BatchNormalization
- Optimized for mel-spectrogram features (128, 431, 1)
- Data augmentation: amplitude scaling, noise, time shift

**Model Comparison:**
Compare this Custom CNN with MobileNetV2 transfer learning:
- **Custom CNN**: Built from scratch, optimized for audio spectrograms
- **MobileNetV2**: Pre-trained on ImageNet, adapted for audio

**Next Steps:**
1. Compare validation accuracy between Custom CNN and MobileNetV2
2. Analyze confusion matrices for both models
3. Choose best performing model for deployment
4. Deploy TFLite model to Raspberry Pi